## Part 1: Preprocessing

In [78]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df[['Age', 'DistanceFromHome', 'HourlyRate', 'JobSatisfaction', 'JobInvolvement', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']].head()

,Age,DistanceFromHome,HourlyRate,JobSatisfaction,JobInvolvement,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
0,41,1,94,4,3,0,1,6,0,8
1,49,8,61,2,2,1,3,10,1,1
2,37,2,92,3,2,0,3,0,0,6
3,33,3,56,3,3,0,3,8,3,1
4,27,2,40,2,3,1,3,2,2,9


In [ ]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [80]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [81]:
# Create a list of at least 10 column names to use as X data
column_list = ['Age', 'DistanceFromHome', 'HourlyRate', 'JobSatisfaction', 'JobInvolvement', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[column_list]
#X_df.head()

# Show the data types for X_df
X_df.dtypes


,0
Age,int64
DistanceFromHome,int64
HourlyRate,int64
JobSatisfaction,int64
JobInvolvement,int64
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [82]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)


In [83]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary


In [84]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)


In [85]:
# Create a OneHotEncoder for the Department column
import sklearn
from sklearn.preprocessing import OneHotEncoder

dept_encoder = OneHotEncoder(handle_unknown='infrequent_if_exist', sparse_output=False)

# Fit the encoder to the training data
dept_encode_train = dept_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept_encoded = dept_encode_train.transform(y_train[['Department']])
y_test_dept_encoded = dept_encode_train.transform(y_test[['Department']])

y_train_dept_encoded

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [86]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(handle_unknown='infrequent_if_exist', sparse_output=False)

# Fit the encoder to the training data
attrition_encode_train = attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_encode_train.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encode_train.transform(y_test[['Attrition']])

y_train_attrition_encoded


array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

## Create, Compile, and Train the Model

In [87]:
# Find the number of columns in the X training data
print(scaled_X_train.shape)

# Create the input layer
input_layer = layers.Input(shape=(10,), name="input_layer")

# Create at least two shared layers
shared_layer1 = layers.Dense(units=64, activation='relu', name="shared_layer1")(input_layer)
shared_layer2 = layers.Dense(units=128, activation='relu', name="shared_layer2")(shared_layer1)


(1102, 10)


In [88]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_branch = layers.Dense(units=32, activation='relu', name="dept_branch")(shared_layer2)

# Create the output layer
dept_output = layers.Dense(3, activation='softmax', name="dept_output")(dept_branch)

dept_output.shape


(None, 3)

In [89]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_branch = layers.Dense(units=32, activation='relu', name="attrition_branch")(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation='softmax', name="attrition_output")(attrition_branch)

attrition_output.shape


(None, 2)

In [90]:
# Create the model
from tensorflow.keras.metrics import BinaryAccuracy, CategoricalAccuracy, Precision, Recall, AUC
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy

model = Model(inputs=input_layer, outputs=[dept_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'dept_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'
                    },
              metrics={
                  'dept_output': [
                      'accuracy',
                      Precision(name='precision_dept'),
                      Recall(name='recall_dept'),
                      AUC(name='auc_dept')
                      ],
                  'attrition_output': [
                      'accuracy',
                      Precision(name='precision_attrition'),
                      Recall(name='recall_attrition'),
                      AUC(name='auc_attrition')
                      ]
              }
              )

# Summarize the model
model.summary()

Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer1 (Dense)     │ (None, 64)             │            704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer2 (Dense)     │ (None, 128)            │          8,320 │ shared_layer1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dept_branch (Dense)       │ (None, 32)             │          4,128 │ shared_layer2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_branch (Dense)  │ (None, 32)             │          4,128 │ shared_layer2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dept_output (Dense)       │ (None, 3)              │             99 │ dept_branch[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_branch[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [91]:
# Train the model
# Ensure that the input data shape matches the model's input layer shape
input_shape = model.layers[0]  # Get the expected input shape
print(f"Expected Input Shape: {input_shape.input}")  # Print the expected shape
print(f"Actual Input Shape: {scaled_X_train.shape}")  # Print the actual shape

model.fit(scaled_X_train,
          {'dept_output': y_train_dept_encoded, 'attrition_output': y_train_attrition_encoded},
          epochs=100,
          shuffle=True,
          verbose=2
          )


Expected Input Shape: []
Actual Input Shape: (1102, 10)
Epoch 1/100


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 2), output.shape=(None, 3)

In [ ]:
# Evaluate the model with the testing data


In [ ]:
# Print the accuracy for both department and attrition


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.